In [4]:
import pandas as pd
import numpy as np
import re
import string
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

In [5]:
# ===== Bagian B: Load Dataset dan Kamus Slang / Stopword =====
df = pd.read_csv("ulasan_goride.csv")
slang_dict = dict(line.strip().split(':') for line in open("kamus_slang.txt", encoding='utf-8'))
stopword_custom = set(open("stopwordsID.txt", encoding='utf-8').read().splitlines())

In [6]:
# ===== Bagian C: Preprocessing =====
print("\n=== Preprocessing Data ===")
factory = StemmerFactory()
stemmer = factory.create_stemmer()

def print_step(step_name, before, after):
    """Utility untuk menampilkan before-after"""
    print(f"\n=== {step_name.upper()} ===")
    print(f"Before: {before}")
    print(f"After:  {after}")


=== Preprocessing Data ===


In [7]:
# Langkah 1: Cleaning Text
print("\n=== Langkah 1: Text Cleaning ===")
def clean_text(text):
    """Fungsi cleaning text"""
    before = text
    after = text.lower()
    after = re.sub(r'http\S+|www\S+', '', after)
    after = re.sub(r'[^a-zA-Z\s]', ' ', after)
    after = re.sub(r'\s+', ' ', after).strip()
    print_step("Text Cleaning", before, after)
    return after

df['after_clean'] = df['ulasan'].astype(str).apply(clean_text)
print("\nHasil setelah cleaning:")
df[['ulasan', 'after_clean']].head(3)


=== Langkah 1: Text Cleaning ===

=== TEXT CLEANING ===
Before: Matikan Fitur Order ganda pemesanan GoRide. Seharusnya kalau sudah Order GoRide tidak bisa order GoRide lagi sekaligus, harus dibatalkan dulu baru bisa pesan lagi
After:  matikan fitur order ganda pemesanan goride seharusnya kalau sudah order goride tidak bisa order goride lagi sekaligus harus dibatalkan dulu baru bisa pesan lagi

=== TEXT CLEANING ===
Before: Tolong ya..Pleeeeease...buat CUSTOMER yg Pesen Gibride....Tolong itu Jempol Jangan Dikit2 cansel....Dikit2 Cansel.....Kalian Beratus Ratus kali Cansel nggak ngaruh ..nggak ngefek ..kitw2 yg driver yg kena ampas nanti Ama aplikator.....Emang Kalian Bayar Tarif nya Super Mahal ...tapi Kita dibpitong Gede super Gede Potongannya.....tolong Kasihani Kami ya Driver Ojol....
After:  tolong ya pleeeeease buat customer yg pesen gibride tolong itu jempol jangan dikit cansel dikit cansel kalian beratus ratus kali cansel nggak ngaruh nggak ngefek kitw yg driver yg kena ampas na

,ulasan,after_clean
0,Matikan Fitur Order ganda pemesanan GoRide. Se...,matikan fitur order ganda pemesanan goride seh...
1,Tolong ya..Pleeeeease...buat CUSTOMER yg Pesen...,tolong ya pleeeeease buat customer yg pesen gi...
2,Saya harap di update kedepanya saat pesan go f...,saya harap di update kedepanya saat pesan go f...


In [8]:
 #Langkah 2: Normalisasi Slang
print("\n=== Langkah 2: Slang Normalization ===")
def normalize_slang(text):
    """Normalisasi slang words"""
    before = text
    words = text.split()
    normalized = [slang_dict.get(word, word) for word in words]
    after = ' '.join(normalized)
    print_step("Slang Normalization", before, after)
    return after

df['after_slang'] = df['after_clean'].apply(normalize_slang)
print("\nHasil setelah normalisasi slang:")
df[['after_clean', 'after_slang']].head(3)


=== Langkah 2: Slang Normalization ===

=== SLANG NORMALIZATION ===
Before: matikan fitur order ganda pemesanan goride seharusnya kalau sudah order goride tidak bisa order goride lagi sekaligus harus dibatalkan dulu baru bisa pesan lagi
After:  matikan fitur order ganda pemesanan goride seharusnya kalau sudah order goride tidak bisa order goride lagi sekaligus harus dibatalkan dulu baru bisa pesan lagi

=== SLANG NORMALIZATION ===
Before: tolong ya pleeeeease buat customer yg pesen gibride tolong itu jempol jangan dikit cansel dikit cansel kalian beratus ratus kali cansel nggak ngaruh nggak ngefek kitw yg driver yg kena ampas nanti ama aplikator emang kalian bayar tarif nya super mahal tapi kita dibpitong gede super gede potongannya tolong kasihani kami ya driver ojol
After:  tolong iya pleeeeease buat customer yang pesan gibride tolong itu jempol jangan dikit cansel dikit cansel kalian beratus ratus kali cansel tidak berpengaruh tidak ngefek kitw yang driver yang kena ampas nanti sam

,after_clean,after_slang
0,matikan fitur order ganda pemesanan goride seh...,matikan fitur order ganda pemesanan goride seh...
1,tolong ya pleeeeease buat customer yg pesen gi...,tolong iya pleeeeease buat customer yang pesan...
2,saya harap di update kedepanya saat pesan go f...,saya harap di update kedepanya saat pesan go f...


In [9]:
# Langkah 3: Remove Repeated Characters
def remove_repeated(text):
    """Reduksi huruf berulang"""
    before = text
    after = re.sub(r'(\w)\1{2,}', r'\1\1', text)
    print_step("Remove Repeated Characters", before, after)
    return after

print("\n=== Langkah 3: Remove Repeated Characters ===")
df['after_repeated'] = df['after_slang'].apply(remove_repeated)
print("\nHasil setelah remove repeated characters:")
df[['after_slang', 'after_repeated']].head(3)


=== Langkah 3: Remove Repeated Characters ===

=== REMOVE REPEATED CHARACTERS ===
Before: matikan fitur order ganda pemesanan goride seharusnya kalau sudah order goride tidak bisa order goride lagi sekaligus harus dibatalkan dulu baru bisa pesan lagi
After:  matikan fitur order ganda pemesanan goride seharusnya kalau sudah order goride tidak bisa order goride lagi sekaligus harus dibatalkan dulu baru bisa pesan lagi

=== REMOVE REPEATED CHARACTERS ===
Before: tolong iya pleeeeease buat customer yang pesan gibride tolong itu jempol jangan dikit cansel dikit cansel kalian beratus ratus kali cansel tidak berpengaruh tidak ngefek kitw yang driver yang kena ampas nanti sama aplikator memang kalian bayar tarif nya super mahal tapi kita dibpitong gede super gede potongannya tolong kasihani kami iya driver ojol
After:  tolong iya pleease buat customer yang pesan gibride tolong itu jempol jangan dikit cansel dikit cansel kalian beratus ratus kali cansel tidak berpengaruh tidak ngefek kitw yang

,after_slang,after_repeated
0,matikan fitur order ganda pemesanan goride seh...,matikan fitur order ganda pemesanan goride seh...
1,tolong iya pleeeeease buat customer yang pesan...,tolong iya pleease buat customer yang pesan gi...
2,saya harap di update kedepanya saat pesan go f...,saya harap di update kedepanya saat pesan go f...


In [10]:
# Langkah 4: Tokenisasi
print("\n=== Langkah 4: Tokenization ===")
def tokenize_text(text):
    """Tokenisasi teks"""
    before = text
    after = text.split()
    print_step("Tokenization", before, after)
    return after

df['after_tokenize'] = df['after_repeated'].apply(tokenize_text)
print("\nHasil setelah tokenization:")
df[['after_repeated', 'after_tokenize']].head(3)


=== Langkah 4: Tokenization ===

=== TOKENIZATION ===
Before: matikan fitur order ganda pemesanan goride seharusnya kalau sudah order goride tidak bisa order goride lagi sekaligus harus dibatalkan dulu baru bisa pesan lagi
After:  ['matikan', 'fitur', 'order', 'ganda', 'pemesanan', 'goride', 'seharusnya', 'kalau', 'sudah', 'order', 'goride', 'tidak', 'bisa', 'order', 'goride', 'lagi', 'sekaligus', 'harus', 'dibatalkan', 'dulu', 'baru', 'bisa', 'pesan', 'lagi']

=== TOKENIZATION ===
Before: tolong iya pleease buat customer yang pesan gibride tolong itu jempol jangan dikit cansel dikit cansel kalian beratus ratus kali cansel tidak berpengaruh tidak ngefek kitw yang driver yang kena ampas nanti sama aplikator memang kalian bayar tarif nya super mahal tapi kita dibpitong gede super gede potongannya tolong kasihani kami iya driver ojol
After:  ['tolong', 'iya', 'pleease', 'buat', 'customer', 'yang', 'pesan', 'gibride', 'tolong', 'itu', 'jempol', 'jangan', 'dikit', 'cansel', 'dikit', 'canse

,after_repeated,after_tokenize
0,matikan fitur order ganda pemesanan goride seh...,"[matikan, fitur, order, ganda, pemesanan, gori..."
1,tolong iya pleease buat customer yang pesan gi...,"[tolong, iya, pleease, buat, customer, yang, p..."
2,saya harap di update kedepanya saat pesan go f...,"[saya, harap, di, update, kedepanya, saat, pes..."


In [11]:
# Langkah 5: Stopword Removal
print("\n=== Langkah 5: Stopword Removal ===")
def remove_stopwords(tokens):
    """Menghapus stopword"""
    if isinstance(tokens, str):  # Handle case if input is string
        tokens = tokens.split()
    before = tokens.copy()
    after = [word for word in tokens if word not in stopword_custom]
    print_step("Stopword Removal", before, after)
    return after

df['after_stopwords'] = df['after_tokenize'].apply(remove_stopwords)
print("\nHasil setelah stopword removal:")
df[['after_tokenize', 'after_stopwords']].head(3)


=== Langkah 5: Stopword Removal ===

=== STOPWORD REMOVAL ===
Before: ['matikan', 'fitur', 'order', 'ganda', 'pemesanan', 'goride', 'seharusnya', 'kalau', 'sudah', 'order', 'goride', 'tidak', 'bisa', 'order', 'goride', 'lagi', 'sekaligus', 'harus', 'dibatalkan', 'dulu', 'baru', 'bisa', 'pesan', 'lagi']
After:  ['matikan', 'fitur', 'order', 'ganda', 'pemesanan', 'goride', 'order', 'goride', 'order', 'goride', 'dibatalkan', 'pesan']

=== STOPWORD REMOVAL ===
Before: ['tolong', 'iya', 'pleease', 'buat', 'customer', 'yang', 'pesan', 'gibride', 'tolong', 'itu', 'jempol', 'jangan', 'dikit', 'cansel', 'dikit', 'cansel', 'kalian', 'beratus', 'ratus', 'kali', 'cansel', 'tidak', 'berpengaruh', 'tidak', 'ngefek', 'kitw', 'yang', 'driver', 'yang', 'kena', 'ampas', 'nanti', 'sama', 'aplikator', 'memang', 'kalian', 'bayar', 'tarif', 'nya', 'super', 'mahal', 'tapi', 'kita', 'dibpitong', 'gede', 'super', 'gede', 'potongannya', 'tolong', 'kasihani', 'kami', 'iya', 'driver', 'ojol']
After:  ['tolong', 

,after_tokenize,after_stopwords
0,"[matikan, fitur, order, ganda, pemesanan, gori...","[matikan, fitur, order, ganda, pemesanan, gori..."
1,"[tolong, iya, pleease, buat, customer, yang, p...","[tolong, iya, pleease, customer, pesan, gibrid..."
2,"[saya, harap, di, update, kedepanya, saat, pes...","[harap, update, kedepanya, pesan, go, food, mu..."


In [12]:
# Langkah 6: Stemming
print("\n=== Langkah 6: Stemming ===")
def stem_tokens(tokens):
    """Stemming token"""
    if isinstance(tokens, str):  # Handle case if input is string
        tokens = tokens.split()
    before = tokens.copy()
    after = [stemmer.stem(word) for word in tokens]
    print_step("Stemming", before, after)
    return after

df['after_stem'] = df['after_stopwords'].apply(stem_tokens)
print("\nHasil setelah stemming:")
df[['after_stopwords', 'after_stem']].head(3)


=== Langkah 6: Stemming ===

=== STEMMING ===
Before: ['matikan', 'fitur', 'order', 'ganda', 'pemesanan', 'goride', 'order', 'goride', 'order', 'goride', 'dibatalkan', 'pesan']
After:  ['mati', 'fitur', 'order', 'ganda', 'mesan', 'goride', 'order', 'goride', 'order', 'goride', 'batal', 'pesan']

=== STEMMING ===
Before: ['tolong', 'iya', 'pleease', 'customer', 'pesan', 'gibride', 'tolong', 'jempol', 'dikit', 'cansel', 'dikit', 'cansel', 'beratus', 'ratus', 'kali', 'cansel', 'berpengaruh', 'ngefek', 'kitw', 'driver', 'kena', 'ampas', 'aplikator', 'bayar', 'tarif', 'nya', 'super', 'mahal', 'dibpitong', 'gede', 'super', 'gede', 'potongannya', 'tolong', 'kasihani', 'iya', 'driver', 'ojol']
After:  ['tolong', 'iya', 'pleease', 'customer', 'pesan', 'gibride', 'tolong', 'jempol', 'dikit', 'cansel', 'dikit', 'cansel', 'atus', 'ratus', 'kali', 'cansel', 'pengaruh', 'ngefek', 'kitw', 'driver', 'kena', 'ampas', 'aplikator', 'bayar', 'tarif', 'nya', 'super', 'mahal', 'dibpitong', 'gede', 'super',

,after_stopwords,after_stem
0,"[matikan, fitur, order, ganda, pemesanan, gori...","[mati, fitur, order, ganda, mesan, goride, ord..."
1,"[tolong, iya, pleease, customer, pesan, gibrid...","[tolong, iya, pleease, customer, pesan, gibrid..."
2,"[harap, update, kedepanya, pesan, go, food, mu...","[harap, update, depa, pesan, go, food, muncul,..."


In [13]:
# Langkah 7: Rejoin Tokens
print("\n=== Langkah 7: Rejoin Tokens ===")
def rejoin_tokens(tokens):
    """Menggabungkan token kembali menjadi teks"""
    if isinstance(tokens, str):  # Handle case if input is string
        return tokens
    before = tokens.copy()
    after = ' '.join(tokens)
    print_step("Rejoin Tokens", before, after)
    return after

df['clean_text'] = df['after_stem'].apply(rejoin_tokens)
print("\nHasil final:")
df[['ulasan', 'clean_text']].head(10)


=== Langkah 7: Rejoin Tokens ===

=== REJOIN TOKENS ===
Before: ['mati', 'fitur', 'order', 'ganda', 'mesan', 'goride', 'order', 'goride', 'order', 'goride', 'batal', 'pesan']
After:  mati fitur order ganda mesan goride order goride order goride batal pesan

=== REJOIN TOKENS ===
Before: ['tolong', 'iya', 'pleease', 'customer', 'pesan', 'gibride', 'tolong', 'jempol', 'dikit', 'cansel', 'dikit', 'cansel', 'atus', 'ratus', 'kali', 'cansel', 'pengaruh', 'ngefek', 'kitw', 'driver', 'kena', 'ampas', 'aplikator', 'bayar', 'tarif', 'nya', 'super', 'mahal', 'dibpitong', 'gede', 'super', 'gede', 'potong', 'tolong', 'kasihan', 'iya', 'driver', 'ojol']
After:  tolong iya pleease customer pesan gibride tolong jempol dikit cansel dikit cansel atus ratus kali cansel pengaruh ngefek kitw driver kena ampas aplikator bayar tarif nya super mahal dibpitong gede super gede potong tolong kasihan iya driver ojol

=== REJOIN TOKENS ===
Before: ['harap', 'update', 'depa', 'pesan', 'go', 'food', 'muncul', 'not

,ulasan,clean_text
0,Matikan Fitur Order ganda pemesanan GoRide. Se...,mati fitur order ganda mesan goride order gori...
1,Tolong ya..Pleeeeease...buat CUSTOMER yg Pesen...,tolong iya pleease customer pesan gibride tolo...
2,Saya harap di update kedepanya saat pesan go f...,harap update depa pesan go food muncul notifik...
3,"Buatku yg tinggal agak jauh dr kota,ini aplika...",buat tinggal kota aplikasi cpat dpat driver pe...
4,Layanan Goride sangat membantu untuk aktifitas...,layan goride bantu aktifitas hari motor baik b...
5,"ini gojek kenapa ya sekarang, saya pakai akun ...",gojek iya pakai akun suka voucher voucher gori...
6,"Kalau terima rider seleksi dulu lah, setiap na...",terima rider seleksi gojek rider nya emosi melulu
7,"Gojek sekarang mahal meskipun pakai voucher, p...",gojek mahal pakai voucher jalan harga pakai vo...
8,cepat dan murah,cepat murah
9,Keren dah murah,keren murah


In [14]:
# Lihat hasil
df[['ulasan', 'clean_text']].head()
# # 7. Simpan Hasil Preprocessing
df.to_csv('ulasan_goride_bersihhh.csv', index=False)